In [1]:
import cv2
import numpy as np
import torch
from depth_anything_v2.dpt import DepthAnythingV2
import vtk
from PIL import Image


xFormers not available
xFormers not available


In [2]:

model = DepthAnythingV2(encoder='vitb', features=128, out_channels=[96, 192, 384, 768])
model.load_state_dict(torch.load('./checkpoints/depth_anything_v2_vitb.pth', map_location='cuda:0')) #cpu


# model = DepthAnythingV2(encoder='vits', features=64, out_channels=[48, 96, 192, 384])
# model.load_state_dict(torch.load('./checkpoints/depth_anything_v2_vits.pth', map_location='cuda:0'))

def pic23d(pic_path,save_path):
    model.cuda().eval()   #   cpu model.eval() 

    raw_img = cv2.imread(pic_path)  
    height, width = raw_img.shape[:2]  
    target_height = 600  
    scale_ratio = target_height / height  
    target_width = int(width * scale_ratio)  
    raw_img = cv2.resize(raw_img, (target_width, target_height))  


    depth = model.infer_image(raw_img) # HxW raw depth map



    depth_normalized = cv2.normalize(depth, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)  
    depth_display = (depth_normalized * 255).astype(np.uint8)  # 转换为8位图像以便显示  
    cv2.imwrite('./depth_map.png', depth_display)  

    fx = 50  # 焦距x  
    fy = 50# 焦距y  
    cx = raw_img.shape[1] / 2.0  # 光心x  
    cy = raw_img.shape[0] / 2.0  # 光心y  
    height, width = raw_img.shape[:2]  
    
    # 推断深度图  
    depth_ = depth_display/255
    # 深度图可能需要归一化或转换到实际的深度值，这里假设它已经是所需的格式  
    
    # 创建一个空的数组来存储空间位置坐标  
    pos = np.zeros((height * width, 3), dtype=np.float32)  
    
    # 遍历深度图的每个像素  
    for v in range(height):  
        for u in range(width):  
            d = depth_[v, u]  # 获取深度值  
            # 计算空间坐标  
            X = (u - cx) * 1/ fx  
            Y = (v - cy)  * 1/ fy  
            Z = d  *10
            # 将坐标存储到pos数组中  
            idx = v * width + u  
            pos[idx] = [X, Y, Z]  
    
    # 现在pos数组包含了深度图上每个点的空间位置坐标


    with open("a.obj","w") as t:
        t.writelines("mtllib my_mtl.mtl"+"\n")
            
        for l in pos:
            L = "v "+str(l[0])+" "+str(l[1])+" "+str(l[2])+"\n"
            t.writelines(L )
        
        w_x = np.asarray([np.linspace(0.0, 1.0, num=width)]*height).reshape(width*height)
        h_y = np.asarray([np.linspace(1.0, 0.0, num=height)]*width).T.reshape(width*height)
        t_array_ = np.stack([w_x,h_y],axis=1)


        for i,j in t_array_:
            L = "vt " +str(i)+" "+str(j) +"\n"
            t.writelines(L )                
    #  
    #          
        t.writelines("\nusemtl my_mtl\n" )              
        b = np.array(range(height * width)).reshape([ height ,width])
        print(b.shape)  
                
        for i in range(1,b.shape[0]-1):
            for j in range(1,b.shape[1]-1):
                p = [ b[i,j] , b[i+1,j],b[i+1,j+1],b[i,j+1]]
        
                L = "f "+str(p[0])+"/"+ str(p[0])+" "+str(p[1])+"/"+ str(p[1])+" "+str(p[2])+"/"+ str(p[2])+" "+str(p[3])+"/"+ str(p[3])+"\n"
                t.writelines(L )
            


    with open("my_mtl.mtl","w") as m:  
        m.writelines(
    f"""
    newmtl my_mtl
    Ka 1 1 1
    Kd 1 1 1
    d 1
    Ns 0
    illum 1
    map_Kd {pic_path}
    """
        )




    def render_and_save_images(obj_file,  output_file, image_size=(800, 800)):
        # 读取模型
        reader = vtk.vtkOBJReader()
        reader.SetFileName(obj_file)
        reader.Update()

        # 创建一个映射器
        mapper = vtk.vtkPolyDataMapper()
        mapper.SetInputConnection(reader.GetOutputPort())

        # 创建一个演员
        actor = vtk.vtkActor()
        actor.SetMapper(mapper)

        # 读取贴图文件
        texture = vtk.vtkTexture()
        texture_reader = vtk.vtkJPEGReader()  # 假设贴图为JPEG格式
        texture_reader.SetFileName(pic_path)  # 根据.mtl文件中的路径设置
        texture_reader.Update()
        texture.SetInputConnection(texture_reader.GetOutputPort())
        
        # 将贴图应用到演员
        actor.SetTexture(texture)

        # 创建渲染窗口
        render_window = vtk.vtkRenderWindow()
        render_window.SetSize(image_size[0], image_size[1])  # 设置渲染窗口大小 

        # 创建左右视角的渲染器
        total_pic_count = 36
        renderers = []
        for i in range(total_pic_count):
            renderer = vtk.vtkRenderer()
            render_window.AddRenderer(renderer)
            renderers.append(renderer)
            # 禁用光照
            renderer.SetAmbient(100.0, 100.0, 100.0)  # 设置环境光
            actor.GetProperty().SetLighting(False)
            # 设置摄像机
            camera = vtk.vtkCamera()
            
            camera.SetFocalPoint(0, 0, 0)
            # if i == 0:  # 左视角
            #     camera.SetPosition(-2, 0, 30)  # 可以根据需要调整位置
            #     camera.SetViewUp(0, -1, 0)
            # else:  # 右视角
            #     camera.SetPosition(2, 0, 30)  # 右视角位置
            #     camera.SetViewUp(0, -1, 0)

      
            camera.SetPosition(-18+i*1, 0, 25)  # 可以根据需要调整位置
            camera.SetViewUp(0, -1, 0)

            renderer.SetActiveCamera(camera)

            # 添加演员到渲染器
            renderer.AddActor(actor)
            renderer.SetBackground(0.0, 0.0, 0.0)  # 背景颜色

            # 渲染
            render_window.Render()

            # 保存图像
            window_to_image_filter = vtk.vtkWindowToImageFilter()
            window_to_image_filter.SetInput(render_window)
            window_to_image_filter.ReadFrontBufferOff()  # 读取后台缓冲区
            window_to_image_filter.Update()

            writer = vtk.vtkPNGWriter()
            writer.SetFileName(save_path.replace("_/",f"{i+1}/"))
        
            writer.SetInputConnection(window_to_image_filter.GetOutputPort())
            writer.Write()

    

    render_and_save_images('a.obj',  'output.png')


# pic_path = "./37.jpg"
# save_path = "./1.png"
# pic23d(pic_path,save_path)

In [ ]:
import cv2
import os

def save_frames_from_video(video_path, output_folder, num_frames_range=[164,251]):
    # 创建输出文件夹
    os.makedirs(output_folder, exist_ok=True)

    # 打开视频文件
    cap = cv2.VideoCapture(video_path)

    # 检查视频是否成功打开
    if not cap.isOpened():
        print("无法打开视频文件")
        return

    frame_count = 0


    for i in range(0,36):
        save_path = f"C:/Users/Administrator/Pictures/blender_gif/{i+1}/"
        os.makedirs(save_path, exist_ok=True)

    #while (frame_count >= num_frames_range[0] and frame_count < num_frames_range[1]) :
    # for i in range(num_frames_range[0],num_frames_range[1]):
    pic_count = 0
    while True:
        ret, frame = cap.read()  # 读取帧
        if not ret:
            break  # 如果没有更多帧，退出循环
        frame_count += 1
        if  (frame_count >= num_frames_range[0] and frame_count < num_frames_range[1]) :
            pic_count+=1

            # 保存帧为图像文件
            frame_filename = os.path.join(output_folder, f'{pic_count}.jpg')
            cv2.imwrite(frame_filename, frame)
            

            save_path = frame_filename.replace("blender_gif/","blender_gif/_/")

            pic23d(frame_filename,save_path)
   



    
    cap.release()  # 释放视频捕获对象
    print(f'已保存 {frame_count} 帧到 {output_folder}')



# 使用示例
video_file = './video.mp4'  # 视频文件路径
output_dir = 'C:/Users/Administrator/Pictures/blender_gif/'    # 输出文件夹路径
save_frames_from_video(video_file, output_dir)

(600, 337)
(600, 337)
(600, 337)
(600, 337)
(600, 337)
(600, 337)


In [5]:
import requests
from PIL import Image
from io import BytesIO

def upload_pic(pic_file_name, save_base="index.bin"):
    im = Image.open(pic_file_name)
    
    # Adjust the image size while maintaining the aspect ratio
    original_width, original_height = im.size
    target_height = 240
    target_width = int((target_height / original_height) * original_width)
    im = im.resize((target_width, target_height), Image.LANCZOS)  # Use LANCZOS filter for better quality

    # Optionally, convert to RGB if the image has an alpha channel
    if im.mode in ("RGBA", "LA") or (im.mode == "P" and "transparency" in im.info):
        im = im.convert("RGB")

    # Save as a binary file (using JPEG with adjustable quality)
    img_byte = BytesIO()
    im.save(img_byte, format='JPEG', quality=99)  # Adjust quality as needed
    img_data = img_byte.getvalue()

    # Write to binary file
    with open(save_base, "wb") as f:
        f.write(img_data)



In [6]:
import os
pic_path = "C:/Users/Administrator/Pictures/blender_gif/"

 
for i in range(1,37):
    for pic in os.listdir(pic_path+"/"+str(i)):
        if ".jpg" in pic or ".png" in pic:
            # print(pic_path+pic)
            if not os.path.exists("D:/move2/"+str(i)+"/"):
                os.mkdir("D:/move2/"+str(i)+"/")
            upload_pic(pic_path+"/"+str(i)+"/"+pic,"D:/move2/"+str(i)+"/"+str(int(pic.split(".")[0]))+".bin")